# Training Monitoring Examples

Agnostic to the framework

Functions to get started with Pycrayon.<br>

Usage
=====
``` 
    expt = get_experiment(name='s2s_lr_0.1')
    # Loss as scalar
    for t in reversed(xrange(1000)):
        expt.add_scalar_dict(
            {
                'loss_mxe/train': t+random.random()*t//5,
                'loss_mxe/dev': 2*t*random.random()+random.random()*t//5,
            },
        )
        time.sleep(0.01)
    # Weights as hist
    expt.add_histogram_value(
        name='affine_1_weights',
        hist=np.random.randn(10000,).tolist(),
        tobuild=True,
    )
```
### Visualise on monitroing
go to monitoring tab



In [ ]:
import random
import socket
import time

import pycrayon
import numpy as np

from collections import namedtuple

CrayonSettings = namedtuple('CrayonSettings', ['host', 'port'])
CRAYON_SETTINGS = CrayonSettings(host='localhost', port='9119')


def get_experiment(name, settings=CRAYON_SETTINGS):
    """Creates a pycrayon experiment object to log data to."""
    experiment_date = datetime.datetime.now().strftime('%Y.%m.%d-%H:%M:%S')
    experiment_name = '{dt}_{host};{name}'.format(
        dt=experiment_date,
        host=socket.gethostname(),
        name=name,
    )
    return get_crayon_client(settings=settings).create_experiment(experiment_name)


def get_crayon_client(settings=CRAYON_SETTINGS):
    return pycrayon.CrayonClient(hostname=settings.host, port=settings.port)


def clear_expts(settings=CRAYON_SETTINGS):
    get_crayon_client(settings=settings).remove_all_experiments()

# Examples of logging scalar and histogram
### Two experiments with different optimizers

In [ ]:
for name in ('s2s,opt=adam', 's2s,opt=adamax'):
    expt = get_experiment(name=name)
    
    # Losses
    for t in reversed(xrange(1000)):
        # Step number is automatically incremented if not provided.
        expt.add_scalar_dict(
            {
                'loss_mxe/train': t+random.random()*t//5,
                'loss_mxe/dev': 2*t*random.random()+random.random()*t//5,
            },
        )
        time.sleep(0.001)
        
    # Visualize a layer's weights and gradients as histograms.
    for i in xrange(10):
        expt.add_histogram_value(
            name='affine_1_weights',
            hist=(np.random.randn(10000,)+i).tolist(),
            tobuild=True,
            # We could also manually set the step number.
            step=1+i,
        )
        # THOU SHALT VISUALIZE YOUR GRADIENTS
        expt.add_histogram_value(
            name='affine_1_grads',
            hist=((np.random.randn(10000,)*0.001)+i).tolist(),
            tobuild=True,
            step=1+i,
        )